In [18]:
import torch 
import torch.nn as nn 
from torch.nn import Sequential,Conv2d, MaxPool2d, Flatten, Linear, ReLU
from torch.nn import  CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader

###############
import torchvision
from torchvision.datasets import MNIST
from torchvision import transforms



In [19]:
##import train_dataset and test_dataset
train_dataset=MNIST(root='../data/',
                    train=True,
                    download=False)

test_dataset=MNIST(root='../data/',
                   train=False,
                   download=True)
## Checking the dataset
img,lbl=train_dataset[0]
print(type(img))
print(img.size)

<class 'PIL.Image.Image'>
(28, 28)


In [20]:
## Let's apply some transformation 

transform=transforms.Compose(transforms=[transforms.ToTensor(),
                                         transforms.Normalize((0.5,), (0.5, ))]) ## How did it reshape to 3D




*Here is what above transformation did:*


1. **transforms.ToTensor()**:

- This transform converts a PIL image or a NumPy array into a PyTorch tensor.
- For an image, it converts the pixel values to the range [0, 1] and changes the shape from *HxWxC (Height x Width x Channels) to CxHxW, which is the standard in PyTorch.*
- For grayscale images like those in the MNIST dataset, it effectively adds a channel dimension, changing the shape from (28, 28) to (1, 28, 28).

2. **transforms.Normalize((0.5,), (0.5,)):**

- This transform normalizes the tensor to have a mean of 0.5 and a standard deviation of 0.5 for each channel.
- Since MNIST is grayscale, there's only one channel. The normalization helps in stabilizing the training process.

In [21]:
##import train_dataset and test_dataset, Now with transformation
train_dataset=MNIST(root='../data/',
                    train=True,
                    transform=transform,
                    download=False)

test_dataset=MNIST(root='../data/',
                   train=False,
                   transform=transform,
                   download=False)

In [22]:
## Need t o apply transformation 
## Checking the dataset
img,lbl=train_dataset[2]
print(type(img))
print(img.shape)

<class 'torch.Tensor'>
torch.Size([1, 28, 28])


In [23]:
batch_size=512
train_dataloader=DataLoader(dataset=train_dataset,
                            shuffle=True,
                            batch_size=batch_size)
test_dataloader=DataLoader(dataset=test_dataset,
                           shuffle=True,
                           batch_size=batch_size)

In [24]:
class Mnist_Classifier(nn.Module):
    def __init__(self,in_channel=1):
        super(Mnist_Classifier,self).__init__()
        self.con1=Conv2d(in_channels=in_channel,
                         out_channels=64,
                         kernel_size=3)
        self.con2=Conv2d(64,32,3)
        self.flatten=Flatten()
        self.linear1=Linear(32*5*5,10)

    def forward(self,x):
        x=self.con1(x)
        x=ReLU()(x)
        x=MaxPool2d(kernel_size=2)(x)
        x=self.con2(x)
        x=ReLU()(x)
        x=MaxPool2d(kernel_size=2)(x)
        x=self.flatten(x)
        x=self.linear1(x)
        return x 




In [25]:
model=Mnist_Classifier().to(device='cuda')

In [26]:
## The model is working correctly for our dataset
model(img.unsqueeze(1).to(device='cuda')).shape

torch.Size([1, 10])

In [27]:
## Find out the error in above 
lr=0.01
opt=SGD(model.parameters(),lr=lr)
criterion=CrossEntropyLoss()
epochs=10


In [28]:
for epoch in range(epochs):

    for step_num,(imgs,lbs) in enumerate(train_dataloader):

        imgs=imgs.to(device='cuda')
        lbs=lbs.to(device='cuda')

        pred_lbs=model(imgs)
        loss=criterion(pred_lbs,lbs)

        loss.backward()

        opt.step()
        opt.zero_grad()

        if step_num%10==0:
            print('Epoch Num:{:03d}  | Step Num:{:03d}'.format(epoch,step_num))
            print('Loss Value: {:.3f}'.format(loss))

        


Epoch Num:000  | Step Num:000
Loss Value: 2.310
Epoch Num:000  | Step Num:010
Loss Value: 2.265
Epoch Num:000  | Step Num:020
Loss Value: 2.214
Epoch Num:000  | Step Num:030
Loss Value: 2.154
Epoch Num:000  | Step Num:040
Loss Value: 2.066
Epoch Num:000  | Step Num:050
Loss Value: 1.985
Epoch Num:000  | Step Num:060
Loss Value: 1.823
Epoch Num:000  | Step Num:070
Loss Value: 1.669
Epoch Num:000  | Step Num:080
Loss Value: 1.482
Epoch Num:000  | Step Num:090
Loss Value: 1.319
Epoch Num:000  | Step Num:100
Loss Value: 1.132
Epoch Num:000  | Step Num:110
Loss Value: 1.006
Epoch Num:001  | Step Num:000
Loss Value: 0.948
Epoch Num:001  | Step Num:010
Loss Value: 0.785
Epoch Num:001  | Step Num:020
Loss Value: 0.750
Epoch Num:001  | Step Num:030
Loss Value: 0.645
Epoch Num:001  | Step Num:040
Loss Value: 0.628
Epoch Num:001  | Step Num:050
Loss Value: 0.620
Epoch Num:001  | Step Num:060
Loss Value: 0.545
Epoch Num:001  | Step Num:070
Loss Value: 0.494
Epoch Num:001  | Step Num:080
Loss Value

In [29]:
model.eval()

Mnist_Classifier(
  (con1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (con2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=800, out_features=10, bias=True)
)

In [35]:
torch.argmax(model(imgs),1)

tensor([7, 4, 7, 8, 9, 2, 2, 1, 5, 5, 7, 6, 9, 9, 1, 5, 9, 0, 8, 8, 1, 2, 4, 0,
        4, 8, 4, 3, 3, 7, 7, 1, 9, 7, 7, 7, 3, 6, 1, 2, 7, 0, 6, 2, 2, 0, 2, 4,
        0, 5, 9, 1, 2, 3, 2, 8, 8, 3, 5, 2, 4, 5, 9, 7, 3, 7, 7, 0, 1, 6, 6, 7,
        2, 9, 6, 9, 9, 6, 8, 6, 4, 9, 4, 1, 4, 8, 8, 5, 6, 1, 2, 2, 6, 7, 5, 1],
       device='cuda:0')